In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

83549

In [3]:
!mkdir pages

In [9]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}page-{i}')

100%|████████████████████████████████████| 83549/83549 [02:02<00:00, 683.20it/s]


In [10]:
urls = sorted(list(set(urls)))

In [11]:
len(urls)

2870

In [16]:
urls

['https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-10',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-11',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-12',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-2',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-3',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-4',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-5',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-6',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-7',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-8',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1660622page-9',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1804424page-10',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1804424page-11',
 'https://c.cari.com.my/forum.php?mod=viewthread&tid=1804424page-12',
 'https://c.cari.com.my/foru

In [14]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    td = soup.find_all('td', {'class': 't_f'})
    td = [str(t) for t in td]
    return td

In [17]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|███████████████████████████████████████████| 58/58 [03:07<00:00,  3.23s/it]


In [18]:
files = glob('threads/*.json')
len(files)

83549

In [23]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████| 83549/83549 [55:47<00:00, 24.96it/s]


In [24]:
files = glob('pages/*.json')
len(files)

58

In [26]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|███████████████████████████████████████████| 58/58 [00:26<00:00,  2.21it/s]


In [27]:
texts = list(set(texts))
len(texts)

14538891

In [28]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|███████████████████████████| 14538891/14538891 [00:26<00:00, 547763.32it/s]


In [4]:
# !zip -r threads.zip threads

In [5]:
# !zip -r pages.zip pages

In [6]:
# !zip -r forums.zip forums